In [2]:
# 데이터 처리하는 라이브러리 사용하기
import pandas as pd

In [3]:
# 오라클 연결, 접속, SQL구문 처리 라이브러리
# 먼저 prompt에서 설치 : conda install -c conda-forge cx_Oracle
# pip은 무조건 최신버전 설치, conda(install -c conda-)로 설치하면 버전 적합성을 체크한 후 설치됨 
import cx_Oracle

In [4]:
# 오라클 연결하기
dsn=cx_Oracle.makedsn('localhost',1521,'xe')    # makedsn : 오라클과 연결하기. 오라클의 주소에 통로 만드는 것
print(dsn)

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))


In [5]:
# 오라클 접속하기
conn=cx_Oracle.connect('house','dbdb',dsn)
conn

<cx_Oracle.Connection to house@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>

In [6]:
# <데이터베이스 사용방법>

In [9]:
# 1. 데이터베이스와 소통할 객체 생성
cursor=conn.cursor()
cursor

<cx_Oracle.Cursor on <cx_Oracle.Connection to house@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>>

In [11]:
# 2. SQL구문을 커서에 실어서 오라클에 전달(execute)하면,
# 3. 오라클이 SQL구문을 처리한 후.

sql = """
select A.prod_code, A.prod_name, B.sel_name, B.sel_id, A.prod_approval, sum(ord_qty) as "판매수량"
From prod A,(select prod_code, prod_name, sel_name, sel_id, prod_approval,ord_num,ord_qty
                from prod, seller, orders
                Where prod_id = sel_id
                And prod_code = ord_code
                AND prod_approval in '승인완료'
                Order by ord_num DESC) B
Where A.prod_code = B.prod_code
Group by A.prod_code, A.prod_name, B.sel_name, B.sel_id, A.prod_approval

"""

cursor.execute(sql)

<cx_Oracle.Cursor on <cx_Oracle.Connection to house@(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=xe)))>>

In [12]:
# 4. 결과값을 커서에 실어서 보내준다.
# 리스트 형태로 모든 데이터 읽어오기..
row = cursor.fetchall()
print(row)
# 전체 리스트 안에 튜플 형태로 행단위로 들어있다.

[('sp123', '화이트 원형테이블', '박바나', 'banana@gmail.com', '승인완료', 8), ('FU123', '[해외] Cheese 미드센츄리 치즈의자 빈티지 인테리어체어', '박바나', 'banana@gmail.com', '승인완료', 1), ('SU123', '[오늘의딜]  MUJI WEEK 입점기념 아크릴수납함 할인!', '김수납', 'soonap@gmail.com', '승인완료', 6), ('KI123', '[오늘의딜] [선착순 5천원쿠폰] 톡 누르면 열리는 감편한 음식물쓰레기통', '심판다', 'panda@gmail.com', '승인완료', 4), ('KI124', '[오늘의집 단독특가] 데일리 주방칼 5종+칼블럭세트 (올스텐/블랙 택1)', '심판다', 'panda@gmail.com', '승인완료', 1), ('FU124', '루나 인테리어 원형 원목 탁상 거울', '박바나', 'banana@gmail.com', '승인완료', 1), ('FU125', '알렉스 1200 높은 수납장 ALEX 3color', '박바나', 'banana@gmail.com', '승인완료', 4), ('DE123', '앙리 마티스 아트 드로잉 / 55 애플 드로잉/', '오브제', 'deco@gmail.com', '승인완료', 5), ('EL123', '[오늘의딜] [단독] 22년 신제품 3단접이 수납식 BLDC 120도 회전 써큘레이터', '물고기', 'fish@gmail.com', '승인완료', 6), ('HO123', '[25%쿠폰] 3겹 순수소프트 롤화장지(27Mx30롤)X2팩 외 모음', '이생활', 'welllife@naver.com', '승인완료', 2), ('FA123', '[맞춤] 화이트 나비주름 암막커튼 3type', '최원단', 'wondan@hanmail.net', '승인완료', 2), ('LI123', '[해외] 북유럽 플로어램프 장스탠드 LED조명', '김태양', 'sun@naver.com', '승인완료', 1), ('FU126

In [13]:
# 컬럼 이름 확인하기
colname=cursor.description
print(colname)
# 데이터베이스의 역할은 끝남 (모든 정보를 다 가져옴) -> 접속을 끊어줘야 함

[('PROD_CODE', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 20, 20, None, None, 0), ('PROD_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 99, 99, None, None, 0), ('SEL_NAME', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 20, 20, None, None, 0), ('SEL_ID', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 20, 20, None, None, 0), ('PROD_APPROVAL', <cx_Oracle.DbType DB_TYPE_VARCHAR>, 4, 12, None, None, 1), ('판매수량', <cx_Oracle.DbType DB_TYPE_NUMBER>, 127, None, None, None, 1)]


In [14]:
# 5. cursor를 닫은 다음, DB와의 연결을 끊어준다
# 커서 반납하기
cursor.close()

In [15]:
# DB와의 접속 끊기
conn.close()

In [16]:
col = []
for i in colname :
    col.append(i[0])
print(col)
# 위의 colname은 리스트형태. 거기에서 0번째 값이 모두 이름이었기 때문에 가져옴

['PROD_CODE', 'PROD_NAME', 'SEL_NAME', 'SEL_ID', 'PROD_APPROVAL', '판매수량']


In [17]:
# pandas를 사용한 데이터 프레임 형식으로 변환
emp = pd.DataFrame(row, columns=col)   # row는 리스트 형태 그 안에 튜플형식으로 존재.
print(emp)

   PROD_CODE                                        PROD_NAME SEL_NAME  \
0      sp123                                        화이트 원형테이블      박바나   
1      FU123                [해외] Cheese 미드센츄리 치즈의자 빈티지 인테리어체어      박바나   
2      SU123                [오늘의딜]  MUJI WEEK 입점기념 아크릴수납함 할인!      김수납   
3      KI123         [오늘의딜] [선착순 5천원쿠폰] 톡 누르면 열리는 감편한 음식물쓰레기통      심판다   
4      KI124         [오늘의집 단독특가] 데일리 주방칼 5종+칼블럭세트 (올스텐/블랙 택1)      심판다   
5      FU124                              루나 인테리어 원형 원목 탁상 거울      박바나   
6      FU125                      알렉스 1200 높은 수납장 ALEX 3color      박바나   
7      DE123                       앙리 마티스 아트 드로잉 / 55 애플 드로잉/      오브제   
8      EL123  [오늘의딜] [단독] 22년 신제품 3단접이 수납식 BLDC 120도 회전 써큘레이터      물고기   
9      HO123           [25%쿠폰] 3겹 순수소프트 롤화장지(27Mx30롤)X2팩 외 모음      이생활   
10     FA123                         [맞춤] 화이트 나비주름 암막커튼 3type      최원단   
11     LI123                        [해외] 북유럽 플로어램프 장스탠드 LED조명      김태양   
12     FU126                호텔식 프리미엄 독